In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pandas_ta as ta
import sys
sys.path.append (r"C:\work\self-research\self-research\codes\codes")
import self_code as sc
import Utils as ut
import glob
import warnings
warnings.filterwarnings("ignore")

In [ ]:
nifty = sc.resample(pd.read_csv(r"D:\Data\Indian\Index\Spot\NIFTY.csv"), '3min' )

In [ ]:
nifty.drop(columns = ['open', 'high', 'low', 'volume'], axis = 1, inplace = True)

In [ ]:
nifty = pd.concat([nifty.between_time('09:15:00', '11:30:00'), nifty.between_time('12:20:00', '15:30:00')])

In [ ]:
nifty = nifty.sort_index()

In [10]:
status = 0
pnl = pd.DataFrame(columns = ['Type', 'Entry Price', 'Entry Time', 'Exit Price', 'Exit Time', 'Points', 'Pnl'])

for row, value in nifty.iterrows():

    symbol_ce = ut.prepareWeeklyOptionsSymbol('NIFTY', ut.priceToATM(value['close'], 'NIFTY'), 'CE', row)
    symbol_pe = ut.prepareWeeklyOptionsSymbol('NIFTY', ut.priceToATM(value['close'], 'NIFTY'), 'PE', row)
    try:
        ce_data = sc.options_simplify(pd.read_csv(r"D:\Data\Indian\Index\Options\nifty options\{}.csv".format(symbol_ce)))
        pe_data = sc.options_simplify(pd.read_csv(r"D:\Data\Indian\Index\Options\nifty options\{}.csv".format(symbol_pe)))
        
        for i in range(3):
            if pe_data.index[i].time() in nifty.index.time:
                pe_data = pe_data[i:]

            if ce_data.index[i].time() in nifty.index.time:
                ce_data = ce_data[i:]

        ce_data = sc.resample_refined(ce_data, '3min')
        pe_data = sc.resample_refined(pe_data, '3min')

        ce_data['10ema'] = ta.ema(ce_data.Close, length = 10)
        pe_data['10ema'] = ta.ema(pe_data.Close, length = 10)
        
        ce_data = ce_data[row:]
        pe_data = pe_data[row:]
        
        # first condition check over here
        if 0 <= ce_data.iloc[0]['Open'] - ce_data.iloc[0]['Low'] <= 1.1:
            status = 1
        elif 0 <= pe_data.iloc[0]['Open'] - pe_data.iloc[0]['Low'] <= 1.1:
            status = -1
        else:
            status = 0
            continue
        
        # second condition check over here
        if status == 1 and (ce_data.iloc[0]['Close'] < ce_data.iloc[0]['10ema'] or ce_data.iloc[0]['Open'] > ce_data.iloc[0]['10ema']):
            status = 0
            continue
        elif status == -1 and (pe_data.iloc[0]['Close'] < pe_data.iloc[0]['10ema'] or pe_data.iloc[0]['Open'] > pe_data.iloc[0]['10ema']):
            status = 0
            continue
        
        # if the size of the candle is > 20 points, no trade
        if status == 1 and abs(ce_data.iloc[0]['Close'] - ce_data.iloc[0]['Open']) > 20:
            status = 0
            continue
        elif status == -1 and abs(pe_data.iloc[0]['Close'] - pe_data.iloc[0]['Open']) > 20:
            status = 0
            continue

        # backtesting starting here
        if status == 1:
            position = 0
            entry_price_ce = 0
            entry_time_ce = 0
            for idx, val in ce_data.iterrows():
                if position == 0:
                    entry_price_ce = val['Close']
                    entry_time_ce = idx
                    position = 1
                    sl = val['Low'] - 5
                    target = entry_price_ce + (2 * (entry_price_ce - sl))
                    continue
                elif position == 1 and val['High'] >= target >= val['Low']:
                    pnl.loc[len(pnl)] = [symbol_ce, entry_price_ce, entry_time_ce, target, idx, (target - entry_price_ce), (target - entry_price_ce) * 50]
                    position = 0
                    break
                elif position == 1 and val['High'] >= sl >= val['Low']:
                    pnl.loc[len(pnl)] = [symbol_ce, entry_price_ce, entry_time_ce, sl, idx, (sl - entry_price_ce), (sl - entry_price_ce) * 50]
                    position = 0
                    break
                elif idx == ce_data.index[-1]:
                    pnl.loc[len(pnl)] = [symbol_ce, entry_price_ce, entry_time_ce, val['Open'], idx, (val['Open'] - entry_price_ce), (val['Open'] - entry_price_ce) * 50]
                    break
        elif status == -1:
            position = 0
            entry_price_pe = 0
            entry_time_pe = 0
            for idx, val in pe_data.iterrows():
                if position == 0:
                    entry_price_pe = val['Close']
                    entry_time_pe = idx
                    position = 1
                    sl = val['Low'] - 5
                    target = entry_price_pe + (2 * (entry_price_pe - sl))
                    continue
                elif position == 1 and val['High'] >= target >= val['Low']:
                    pnl.loc[len(pnl)] = [symbol_pe, entry_price_pe, entry_time_pe, target, idx, (target - entry_price_pe), (target - entry_price_pe) * 50]
                    position = 0
                    break
                elif position == 1 and val['High'] >= sl >= val['Low']:
                    pnl.loc[len(pnl)] = [symbol_pe, entry_price_pe, entry_time_pe, sl, idx, (sl - entry_price_pe), (sl - entry_price_pe) * 50]
                    position = 0
                    break
                elif idx == pe_data.index[-1]:
                    pnl.loc[len(pnl)] = [symbol_pe, entry_price_pe, entry_time_pe, val['Open'], idx, (val['Open'] - entry_price_pe), (val['Open'] - entry_price_pe) * 50]
                    break


    except:
        continue

In [ ]:
sc.final(pnl, r'C:\\work\\balysis\\Client Backtest\\Shailesh kumar\\10-ema strategy.xlsx', capital=50000)

In [2]:
df = pd.read_csv('pnl.csv')

In [5]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [7]:
df['Entry Time'] = pd.to_datetime(df['Entry Time'])

In [9]:
df['Exit Time'] = pd.to_datetime(df['Exit Time'])

In [11]:
sc.final(df, r'C:\work\balysis\Client Backtest\Shailesh kumar\result.xlsx', capital = 50000)